# Trimming Whitespace

This is a short example profiling some of the various alternatives for trimming whitespace when uploading columns. This example was designed to study the impacts of addressing easyaccess [Issue #52](https://github.com/mgckind/easyaccess/issues/52).

The final result is that the simplest and best performant solution is to add a Oracle `TRIM` operation before the inserted string values. This does not need special treatment for pandas/numpy, and does not have the overhead of doing the whitespace trimming locally (though I'm not sure how Oracle is doing it so fast on the other end...). In the end, the change looks somthing like this:
```sql
INSERT INTO MYTABLE ( TEXT ) values ( TRIM(TRAILING FROM :TEXT) )
```

Because I was running this notebook on my laptop using a wireless connectiong, the timing information is not especially useful (you'll need to trust that 

In [1]:
import numpy as np
import pandas as pd
import fitsio
import time
import copy
import easyaccess as ea


In [2]:
tablename='TMP'
col = 'TAG'

print "Loading FITS data..."
d = fitsio.read('infile.fits',columns=[col])
ds = copy.deepcopy(d)

print "Converting to pandas..."
p = pd.DataFrame(d)
ps = copy.deepcopy(p)

print "Found %s rows.\n"%len(d)
print "Data column and dtype: ",d.dtype.descr
print "Data value (note trailing space): ",np.unique(d[col])

Loading FITS data...
Converting to pandas...
Found 7136105 rows.

Data column and dtype:  [('TAG', '|S13')]
Data value (note trailing space):  ['Y2N_FIRSTCUT ']


In [3]:
print "Creating easyaccess connection..."
con = ea.connect()
try: con.drop_table(tablename)
except: pass

print "Creating table..."
query = "CREATE TABLE %s ( %s VARCHAR2(13) );"%(tablename,col)
print query
con.onecmd(query)

Creating easyaccess connection...
Connecting to DB ** dessci ** ...
Creating table...
CREATE TABLE TMP ( TAG VARCHAR2(13) );
Done!



In [4]:
# numpy strip and insert

start = time.time()
print "Inserting numpy (NO STRIP) into table..."
query = "INSERT INTO %s ( %s ) values (:%s )"%(tablename,col,col)
print query
con.cursor().executemany(query, d.tolist())
print "Runtime: %.2fs"%(time.time() - start)

start = time.time()
print "Inserting numpy (STRIP) into table..."
query = "INSERT INTO %s ( %s ) values (:%s )"%(tablename,col,col)
print query
ds[col] = np.char.strip(ds[col])
con.cursor().executemany(query, d.tolist())
print "Runtime: %.2fs"%(time.time() - start)

Inserting numpy (NO STRIP) into table...
INSERT INTO TMP ( TAG ) values (:TAG )
Runtime: 12.10s
Inserting numpy (STRIP) into table...
INSERT INTO TMP ( TAG ) values (:TAG )
Runtime: 9.64s


In [5]:
# pandas strip and insert

start = time.time()
print "Inserting pandas (NO STRIP) into table..."
query = "INSERT INTO %s ( %s ) values (:%s )"%(tablename,col,col)
print query
con.cursor().executemany(query, p.values.tolist())
print "Runtime: %.2fs"%(time.time() - start)

start = time.time()
print "Inserting pandas (STRIP) into table..."
query = "INSERT INTO %s ( %s ) values (:%s )"%(tablename,col,col)
print query
ps[col] = ps[col].str.strip()
con.cursor().executemany(query, ps.values.tolist())
print "Runtime: %.2fs"%(time.time() - start)

Inserting pandas (NO STRIP) into table...
INSERT INTO TMP ( TAG ) values (:TAG )
Runtime: 15.43s
Inserting pandas (STRIP) into table...
INSERT INTO TMP ( TAG ) values (:TAG )
Runtime: 19.10s


In [6]:
# oracle strip and numpy insert

start = time.time()
print "Inserting numpy (TRIM) into table..."
query = "INSERT INTO %s ( %s ) values ( TRIM(TRAILING FROM :%s) )"%(tablename,col,co\
l)
print query
con.cursor().executemany(query, d.tolist())
print "Runtime: %.2fs"%(time.time() - start)

Inserting numpy (TRIM) into table...
INSERT INTO TMP ( TAG ) values ( TRIM(TRAILING FROM :TAG) )
Runtime: 11.02s


In [7]:
# Now check that the Oracle strip worked

try: con.drop_table(tablename)
except: pass
print "Creating table..."
query = "CREATE TABLE %s ( %s VARCHAR2(13) );"%(tablename,col)
print query
con.onecmd(query)

print
start = time.time()
print "Inserting numpy (oracle) into table..."
query = "INSERT INTO %s ( %s ) values ( TRIM(:%s) )"%(tablename,col,col)
print query
con.cursor().executemany(query, d.tolist())
print "Runtime: %.2fs"%(time.time() - start)

print "Total number of rows..."
query = "select count(*) from %s;"%(tablename)
print query
con.onecmd(query)

print "Selecting original string..."
query = "select count(*) from %s where %s = '%s';"%(tablename,col,d[col][0])
print query
con.onecmd(query)

print "Selecting stripped string..."
query = "select count(*) from %s where %s = '%s';"%(tablename,col,d[col][0].strip())
print query
con.onecmd(query)

Creating table...
CREATE TABLE TMP ( TAG VARCHAR2(13) );
Done!


Inserting numpy (oracle) into table...
INSERT INTO TMP ( TAG ) values ( TRIM(:TAG) )
Runtime: 11.23s
Total number of rows...
select count(*) from TMP;


1 rows in 3.49 seconds

   COUNT(*)
1   7136105

Selecting original string...
select count(*) from TMP where TAG = 'Y2N_FIRSTCUT ';


1 rows in 1.55 seconds

   COUNT(*)
1         0

Selecting stripped string...
select count(*) from TMP where TAG = 'Y2N_FIRSTCUT';


1 rows in 0.33 seconds

   COUNT(*)
1   7136105

